# conf.getAll()

In [10]:
cd ores_bias_project

/srv/home/nathante/ores_bias_project


In [2]:
from datetime import timezone
from datetime import datetime
import pandas as pd
import numpy as np
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql import Window
from functools import reduce
from pyspark.sql.functions import udf
from dateutil import parser
import pickle

In [3]:
spark.sparkContext.addPyFile(".local/lib/python3.5/site-packages/mwcomments-0.2.0-py3.5.egg")
spark.sparkContext.addPyFile(".local/lib/python3.5/site-packages/sortedcontainers-2.1.0-py3.5.egg")
spark.sparkContext.addPyFile(".local/lib/python3.5/site-packages/python_dateutil-2.8.0-py3.5.egg")

In [4]:
import mwcomments

In [5]:
wtm = mwcomments.WikiToolMap.load_WikiToolMap()

In [6]:
broad_wtm = sc.broadcast(wtm)

In [7]:
treated_wikis = pd.read_csv("ores_bias_data/rcfilters_enabled.csv")

In [9]:
# select canonical cutoffs
cutoffs = treated_wikis.groupby(treated_wikis.Wiki).min()

In [10]:
cutoffs.timestamp = pd.to_datetime(cutoffs.timestamp)

In [11]:
cutoffs = cutoffs.reset_index()

In [7]:
# ok we're ready to fire up spark and make a stratified sample
wmhist = spark.read.table("wmf.mediawiki_history")
# we only need the latest snapshot
wmhist = wmhist.filter(f.col("snapshot") == "2019-04")
wmhist = wmhist.filter((f.col("event_entity") == "revision"))

In [8]:
wmhist = wmhist.select(['wiki_db','event_timestamp','event_comment','revision_id','revision_parent_id','revision_text_bytes','revision_text_bytes_diff','revision_text_sha1','revision_is_identity_reverted','revision_first_identity_reverting_revision_id','revision_is_identity_revert','revision_tags','event_user_id','event_user_text','event_user_is_anonymous','event_user_creation_timestamp','event_user_first_edit_timestamp','event_user_revision_count','event_user_seconds_since_previous_revision','page_id','page_title_historical','page_title',                'page_namespace','page_is_redirect','page_is_deleted','page_revision_count','page_seconds_since_previous_revision',"event_user_groups","event_user_is_bot_by","revision_deleted_parts",'revision_deleted_parts_are_suppressed'])


In [14]:
treated_wikis = set(treated_wikis.Wiki)

wmhist = wmhist.withColumn("treated",f.col("wiki_db").isin(treated_wikis))

wmhist = wmhist.filter(f.col("treated") == True)


In [17]:
missing_configs = wmhist.filter( (f.col("page_title").like("%Undo-summary%")) | (f.col("page_title").like("%Revertpage%"))).filter(f.col("page_is_deleted")).select('wiki_db','revision_id','page_id','page_title').toPandas()

missing_configs.to_csv("deleted_config_revisions_treated.csv")

Py4JJavaError: An error occurred while calling o196.collectToPython.
: org.apache.spark.SparkException: Job 1 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1602)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1542)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1789)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1772)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1761)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:642)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2034)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2055)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2074)
	at org.apache.spark.S

In [ ]:
import json

In [11]:
from spark_functions import build_wmhist_step1
wmhist = build_wmhist_step1(wmhist)


In [12]:
from spark_functions import process_reverts
reverts = process_reverts(wmhist)

In [19]:
testpd = wmhist.filter( (f.col("wiki_db") == 'enwiki') & 
              (f.array_contains(col="revision_tags",value="mw-undo")) & 
              (f.array_contains(col="comment_match",value="undo"))).select(['event_comment',
                                                                            'comment_match',
                                                                             'revision_tags',
                                                                            'revision_id',
                                                                            'event_timestamp']).limit(30).toPandas()

Py4JJavaError: An error occurred while calling o273.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 3.0 failed 4 times, most recent failure: Lost task 0.3 in stage 3.0 (TID 1963, analytics1073.eqiad.wmnet, executor 27): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark2/python/pyspark/worker.py", line 219, in main
    func, profiler, deserializer, serializer = read_udfs(pickleSer, infile, eval_type)
  File "/usr/lib/spark2/python/pyspark/worker.py", line 139, in read_udfs
    arg_offsets, udf = read_single_udf(pickleSer, infile, eval_type)
  File "/usr/lib/spark2/python/pyspark/worker.py", line 119, in read_single_udf
    f, return_type = read_command(pickleSer, infile)
  File "/usr/lib/spark2/python/pyspark/worker.py", line 59, in read_command
    command = serializer._read_with_length(file)
  File "/usr/lib/spark2/python/pyspark/serializers.py", line 170, in _read_with_length
  

In [18]:
wmhist = wmhist.filter(f.col("wiki_db").isin(list(treated_wikis)))

In [19]:
# for convenience we'll use the start of the week as the time of intervention and ignore the fact that it was actually mid-week in some cases
wmhist = wmhist.withColumn("week",f.date_trunc("week",wmhist.event_timestamp))

In [20]:
min_time = f.lit("2007-01-01")
wmhist = wmhist.filter(f.col('week') > min_time)

In [21]:
edits_by_user_week = wmhist.filter(f.col("page_namespace")==0).groupBy(['wiki_db','week','event_user_id']).agg(f.count(wmhist.revision_id).alias("wiki_week_counts"))

In [22]:
wikis = wmhist.select("wiki_db").distinct().toPandas()
weeks = wmhist.select("week").distinct().toPandas()

Py4JJavaError: An error occurred while calling o251.collectToPython.
: org.apache.spark.SparkException: Job 0 cancelled 
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:1602)
	at org.apache.spark.scheduler.DAGScheduler.handleJobCancellation(DAGScheduler.scala:1542)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.doOnReceive(DAGScheduler.scala:1789)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1772)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onReceive(DAGScheduler.scala:1761)
	at org.apache.spark.util.EventLoop$$anon$1.run(EventLoop.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.runJob(DAGScheduler.scala:642)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2034)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2055)
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2074)
	at org.apache.spark.S

In [ ]:
wikis.wiki_db.to_csv("all_wikis.csv",index=False)

In [ ]:
wikis = wikis.assign(key=1)
weeks = weeks.assign(key=1)
wiki_weeks = wikis.merge(weeks,on='key')
wiki_weeks = wiki_weeks.drop('key',axis=1)

In [ ]:
wiki_weeks = spark.createDataFrame(wiki_weeks)

In [40]:
# spot check
#wmhist.filter((wmhist.event_user_text == "AaronSw") & (wmhist.wiki_db == "enwiki")).select(['event_user_id','week','wiki_db','event_user_text']).show()

In [41]:
# checks out
###edits_by_user_week.filter((edits_by_user_week.event_user_id==20842) & (edits_by_user_week.wiki_db=="enwiki")).orderBy(edits_by_user_week.week).show()

In [42]:
edits_by_user_week = edits_by_user_week.withColumn("week_unix", f.unix_timestamp(edits_by_user_week.week))

In [43]:
tmin_d28_in_s = (28+7)*24*60*60
tmin_d7_in_s =  (7)*24*60*60

active_window = Window.partitionBy(['wiki_db','event_user_id']).orderBy(['week_unix']).rangeBetween(-1 * tmin_d28_in_s,-1*tmin_d7_in_s)

edits_by_user_week = edits_by_user_week.withColumn("user_edits_last_month",f.sum("wiki_week_counts").over(active_window))
edits_by_user_week = edits_by_user_week.withColumn("is_active", (~ f.isnull(edits_by_user_week.user_edits_last_month)) &   (edits_by_user_week.user_edits_last_month >= 5))

In [44]:
# spot check
#edits_by_user_week.filter((edits_by_user_week.event_user_id==20842) & (edits_by_user_week.wiki_db=='enwiki')).show()

In [45]:
edits_by_week = edits_by_user_week.groupBy(['wiki_db','week']).agg(
    f.sum("wiki_week_counts").alias("total_edits"),
    f.sum(edits_by_user_week.is_active.cast(IntegerType())).alias("active_editors")
)

In [46]:
# break reverts out into: undo, huggle, bot 

In [47]:
#wmhist.filter((f.col("wiki_db")=="aawiki") & (f.col("week") == "2007-11-19 00:00:00") &( wmhist.revision_is_identity_revert == True)).show()

In [48]:
wiki_weeks = wiki_weeks.fillna(0,["mean_user_reverts","n_reverts"])

In [49]:
#wiki_weeks.filter(f.isnull(f.col("user_week_sd"))).show()

In [51]:
#wmhist.filter(~f.isnull(f.col("revision_tags"))).select(f.min(f.col("week"))).show()

In [52]:
wmhist = wmhist.withColumn(
    'revert_tool_tag',
    f.when(
        f.array_contains(f.col("revision_tags"),"mw-undo"),'undo').otherwise(
        f.when(
            f.array_contains(f.col("revision_tags"),"mw-rollback"),'rollback').otherwise(                             f.when(f.array_contains(f.col("revision_tags"),'huggle'),'huggle').otherwise(
            f.when(f.array_contains(f.col("revision_tags"),"twinkle"),"twinkle").otherwise(
            "otherTool")))))


In [18]:
wmhist = wmhist.withColumn("revert_tools_match", match_comment(f.col("event_comment"),f.col("wiki_db"),f.col('event_timestamp')))

NameError: name 'match_comment' is not defined

In [17]:
wmhist = wmhist.withColumn(
    'revert_tool',
    f.when(
        f.array_contains(f.col("revert_tools_match"),"undo"),'undo').otherwise(
        f.when(
            f.array_contains(f.col("revert_tools_match"),"rollback"),'rollback').otherwise(                             f.when(f.array_contains(f.col("revert_tools_match"),'huggle'),'huggle').otherwise(
            f.when(f.array_contains(f.col("revert_tools_match"),"twinkle"),"twinkle").otherwise(
            "otherTool")))))

AnalysisException: "cannot resolve '`revert_tools_match`' given input columns: [mediawiki_history.revision_parent_id, mediawiki_history.revision_deleted_parts_are_suppressed, mediawiki_history.revision_deleted_parts, mediawiki_history.page_seconds_since_previous_revision, mediawiki_history.page_revision_count, mediawiki_history.page_title_historical, mediawiki_history.page_namespace, mediawiki_history.event_user_revision_count, event_user_isbot2, mediawiki_history.wiki_db, event_user_isadmin, mediawiki_history.revision_text_bytes_diff, mediawiki_history.event_user_seconds_since_previous_revision, week, comment_match, role_type, mediawiki_history.page_is_deleted, mediawiki_history.event_user_text, anon_new_established, mediawiki_history.event_user_first_edit_timestamp, is_undo, is_rollback, mediawiki_history.event_timestamp, mediawiki_history.event_user_id, mediawiki_history.revision_id, mediawiki_history.event_user_creation_timestamp, event_user_ispatroller, mediawiki_history.revision_

In [ ]:
# we only want to look at the damaging reverts for measuring our vandalism fighting outcomes

reverts = reverts.filter(f.col("is_damage") == True)      
# exclude reverts with ttr > 30 days = 60 seconds * 60 minutes / second * 24hours / day * 30 days           
reverts = reverts.filter(f.col("time_to_revert") <= 30*24*60*60)                                                                                                                                               
reverts = reverts.withColumn("med_ttr", f.expr('percentile_approx(time_to_revert, 0.5,1)').over(Window.partitionBy(['wiki_db','week'])))                                                                       

In [56]:
# crap the tags only exist starting in 2018

In [57]:
#wmhist.filter( (f.col("revert_tool")=="undo") & f.col("wiki_db") == "enwiki").select(f.min(f.col("week"))).show()

In [58]:
# the spike in reverts is real

In [59]:
# next lets look at time to revert
reverteds = wmhist.filter((wmhist.page_namespace==0) & (wmhist.revision_is_identity_reverted == True)).select(['wiki_db','event_user_text','revision_id','event_timestamp','revision_first_identity_reverting_revision_id','anon_new_established'])

reverteds = reverteds.withColumnRenamed("event_timestamp","reverted_timestamp")
reverteds = reverteds.withColumnRenamed("revision_id","reverted_revision_id")
reverteds = reverteds.withColumnRenamed("event_user_text","reverted_user_text")

reverts = wmhist.filter((wmhist.page_namespace==0)&(wmhist.revision_is_identity_revert == True)).select(['wiki_db','week','event_user_text','event_timestamp','role_type','revision_id','revision_is_identity_reverted','revision_first_identity_reverting_revision_id','revert_tool','is_undo','is_rollback'])

reverts = reverts.withColumnRenamed("event_user_text","revert_user_text")
reverts = reverts.withColumnRenamed("event_timestamp","revert_timestamp")
reverts = reverts.withColumnRenamed("revision_id","revert_revision_id")
reverts = reverts.withColumnRenamed("wiki_db","wiki_db_l")
reverts = reverts.withColumnRenamed("revision_is_identity_reverted","revert_is_identity_reverted")
reverts = reverts.withColumnRenamed("revision_first_identity_reverting_revision_id","revert_first_identity_reverting_revision_id")

In [60]:
# exclude self-reverts
reverts = reverts.join(reverteds, on=[reverts.wiki_db_l == reverteds.wiki_db, reverts.revert_revision_id == reverteds.revision_first_identity_reverting_revision_id, reverts.revert_user_text != reverteds.reverted_user_text],how='inner')

In [61]:
reverted_reverts = reverts.filter(f.col('revert_is_identity_reverted')==True)

In [62]:
reverted_reverts = reverted_reverts.withColumnRenamed("revert_timestamp","rr_timestamp")
reverted_reverts = reverted_reverts.withColumnRenamed("revert_revision_id","rr_revision_id")
reverted_reverts = reverted_reverts.withColumnRenamed("revert_first_identity_reverting_revision_id","rr_reverting_revision_id")
reverted_reverts = reverted_reverts.withColumnRenamed("wiki_db",'rr_wiki_db')
reverted_reverts = reverted_reverts.withColumnRenamed("week",'rr_week')
reverted_reverts = reverted_reverts.select(['rr_wiki_db','rr_timestamp','rr_revision_id','rr_reverting_revision_id','rr_week'])

In [63]:
reverts = reverts.join(reverted_reverts, on=[reverts.reverted_revision_id == reverted_reverts.rr_revision_id, reverts.wiki_db == reverted_reverts.rr_wiki_db],how='left_outer')

In [65]:
reverts = reverts.withColumn("rr_ttr", (f.unix_timestamp(reverts.revert_timestamp) - f.unix_timestamp(reverts.rr_timestamp)) / 1000)

In [66]:
# spot check
reverts.filter(f.isnull(f.col("rr_ttr")) == False).filter(test_filter).select(['rr_revision_id','reverted_revision_id','rr_reverting_revision_id','revert_revision_id','wiki_db','rr_timestamp','revert_timestamp','reverted_timestamp','rr_ttr'])

DataFrame[rr_revision_id: bigint, reverted_revision_id: bigint, rr_reverting_revision_id: bigint, revert_revision_id: bigint, wiki_db: string, rr_timestamp: string, revert_timestamp: string, reverted_timestamp: string, rr_ttr: double]

In [67]:
reverts = reverts.withColumn("is_damage",f.when(f.isnull(f.col("rr_ttr")), True).otherwise(f.col("rr_ttr") >= 48*60*60))

In [68]:
# we only want to look at the damaging reverts for measuring our vandalism fighting outcomes
reverts = reverts.filter(f.col("is_damage") == True)
# convert time to revert into seconds
reverts = reverts.withColumn("time_to_revert",(f.unix_timestamp(f.col("revert_timestamp")) - f.unix_timestamp(f.col("reverted_timestamp"))) / 1000)
# let's use median ttr as the metric

# exclude reverts with ttr > 30 days = 60 seconds * 60 minutes / second * 24hours / day * 30 days
reverts = reverts.filter(f.col("time_to_revert") <= 30*24*60*60)

reverts = reverts.withColumn("med_ttr", f.expr('percentile_approx(time_to_revert, 0.5,1)').over(Window.partitionBy(['wiki_db','week'])))                                                                      

In [69]:
# for the distribution of reverting activity, we need the counts of reverts by user, wiki, and week
reverts_by_user_week = reverts.groupby(['wiki_db',"revert_user_text","week"]).agg(f.sum(f.when( (f.col("is_damage") == True),1).otherwise(0)).alias("user_week_reverts"))
                                                               

In [70]:
gb = reverts_by_user_week.groupby(['wiki_db','week']).agg(f.sum('user_week_reverts').alias("wiki_week_reverts"),
                                                         f.mean('user_week_reverts').alias("mean_user_reverts"))

In [71]:
reverts_by_user_week = reverts_by_user_week.join(gb,on=["wiki_db","week"])

In [72]:
reverts_by_user_week = reverts_by_user_week.withColumn("p_reverts",f.col("user_week_reverts")/f.col("wiki_week_reverts"))

reverts_by_user_week = reverts_by_user_week.withColumn("p_reverts_sq",f.col("p_reverts") * f.col("p_reverts"))

In [73]:
reverts_by_user_week = reverts_by_user_week.withColumn("user_week_deviance", reverts_by_user_week.user_week_reverts - reverts_by_user_week.mean_user_reverts)

reverts_by_user_week = reverts_by_user_week.withColumn("user_week_var_part", reverts_by_user_week.user_week_deviance * reverts_by_user_week.user_week_deviance)

In [74]:
reverts_by_week = reverts_by_user_week.groupby(["wiki_db","week"]).agg(f.mean("user_week_reverts"). \
                                                                       alias("mean_user_reverts"),
                                                                       f.sum("user_week_reverts"). \
                                                                       alias("n_reverts"),
                                                                       f.mean("user_week_var_part"). \
                                                                       alias("user_week_var"),
                                                                       f.sum("p_reverts_sq"). \
                                                                       alias("revert_hhi"))

In [75]:
reverts_by_week = reverts_by_week.withColumn("user_week_sd",f.sqrt(reverts_by_week.user_week_var))

In [76]:
reverts_by_week = reverts_by_week.withColumn("user_week_revert_cv",reverts_by_week.user_week_sd / reverts_by_week.mean_user_reverts)

In [77]:
wiki_weeks = wiki_weeks.join(reverts_by_week,on=['wiki_db','week'],how='left_outer')

In [78]:
wiki_weeks = wiki_weeks.join(edits_by_week,on=['wiki_db','week'], how='left_outer')

In [79]:
# spot check
#reverts.filter(f.col("is_damage") == False).filter(f.col("wiki_db")=="enwiki").show()

In [80]:
time_to_revert_gb = reverts.groupby(['wiki_db','week'])
time_to_revert = time_to_revert_gb.agg(
    f.mean(f.col("time_to_revert")).alias("mean_ttr"),
    f.stddev(f.col("time_to_revert")).alias("sd_ttr"),
    f.exp(f.mean(f.log(f.col("time_to_revert")))).alias("geom_mean_ttr"),
    f.first(f.col("med_ttr")).alias("med_ttr"),
    f.count(f.col("week")).alias("N_revert"))



In [81]:
wiki_weeks = wiki_weeks.join(time_to_revert,on=['wiki_db','week'],how='left_outer')

In [82]:
# so we see that the count in the main table can be wrong. Why?

In [83]:
wiki_weeks = wiki_weeks.fillna(0,["N_reverteds","active_editors","total_edits"])

In [84]:
#wiki_weeks.filter(f.isnull(f.col('geom_mean_ttr'))).show()

In [85]:
reverts = reverts.withColumnRenamed("role_type","revert_role_type")


In [86]:
wiki_week_tools = reverts.groupby(["wiki_db","week"]).pivot("revert_tool",['undo','rollback','huggle','twinkle','otherTool']).agg(f.mean(f.col("time_to_revert")).alias("mean_ttr"),
    f.stddev(f.col("time_to_revert")).alias("sd_ttr"),
    f.exp(f.mean(f.log(f.col("time_to_revert")))).alias("geom_mean_ttr"),
    f.first(f.col("med_ttr")).alias("med_ttr"),
    f.count("reverted_revision_id").alias("N_reverts"))

In [87]:

wiki_week_roles = reverts.groupby(["wiki_db","week"]).pivot("revert_role_type",['admin','bot','other','patroller']).agg(f.mean(f.col("time_to_revert")).alias("mean_ttr"),
    f.stddev(f.col("time_to_revert")).alias("sd_ttr"),
    f.exp(f.mean(f.log(f.col("time_to_revert")))).alias("geom_mean_ttr"),
    f.first(f.col("med_ttr")).alias("med_ttr"),
    f.count("reverted_revision_id").alias("N_reverts"))

In [88]:
wiki_week_editortypes = reverts.groupby(["wiki_db","week"]).pivot("anon_new_established",['anonymous','newcomer','established']).agg(f.mean(f.col("time_to_revert")).alias("mean_ttr"),
    f.stddev(f.col("time_to_revert")).alias("sd_ttr"),
    f.exp(f.mean(f.log(f.col("time_to_revert")))).alias("geom_mean_ttr"),
    f.first(f.col("med_ttr")).alias("med_ttr"),
    f.count("reverted_revision_id").alias("N_reverts"))

In [89]:
wiki_weeks = wiki_weeks.join(wiki_week_tools, on=['wiki_db','week'],how='left_outer')

In [90]:
wiki_weeks = wiki_weeks.join(wiki_week_roles, on=['wiki_db','week'],how='left_outer')

In [91]:
wiki_weeks = wiki_weeks.join(wiki_week_editortypes, on=['wiki_db','week'],how='left_outer')

In [92]:
reverts = reverts.withColumn('editortype_x_reverttool', f.concat_ws('_',f.col('anon_new_established'),f.col("revert_tool")))

In [93]:
pivot_values = [ed+'_'+tool for ed in ['anonymous','newcomer','established'] for tool in ['undo','rollback','huggle','twinkle','otherTool']]

In [94]:
wiki_week_editortypesxtools = reverts.groupby(["wiki_db","week"]).pivot("editortype_x_reverttool",pivot_values).agg(
    f.exp(f.mean(f.log(f.col("time_to_revert")))).alias("geom_mean_ttr"),
    f.first(f.col("med_ttr")).alias("med_ttr"),
    f.count("reverted_revision_id").alias("N_reverts"))

In [95]:
wiki_weeks = wiki_weeks.join(wiki_week_editortypesxtools, on=['wiki_db','week'],how='left_outer')

In [96]:
interesting_namespaces = [0,1,2,3,4]

In [97]:
tm_vars = wmhist.withColumn("user_new_anon",f.when(wmhist.event_user_is_anonymous,"anon").otherwise(
    f.when(wmhist.event_user_is_newcomer,"newcomer").otherwise("non_anon_newcomer")))

tm_vars = tm_vars.filter(tm_vars.page_namespace.isin(interesting_namespaces)).withColumn("user_new_anon_namespace",f.concat_ws('_',tm_vars.page_namespace,tm_vars.user_new_anon))


In [98]:
tm_vars = tm_vars.groupby(['wiki_db','week']) \
                .pivot("user_new_anon_namespace") \
                .agg(f.count(wmhist.revision_id).alias("N_edits"),
                     f.countDistinct(wmhist.event_user_text).alias("N_editors"))

In [99]:
#tm_vars.show()

In [100]:
wiki_weeks = wiki_weeks.join(tm_vars,on=['wiki_db','week'],how='left_outer')

In [101]:
wiki_weeks = wiki_weeks.withColumn("year",f.year(f.col('week')))
wiki_weeks = wiki_weeks.withColumn("month",f.month(f.col('week')))

In [102]:
#wiki_weeks.show()

In [103]:
#wiki_weeks.write.parquet("/user/nathante/ores_bias/wiki_weeks.parquet",partitionBy=["wiki_db",'year','month'],mode='overwrite')

In [104]:
import re
import requests
from itertools import chain
response = requests.api.get("https://meta.wikimedia.org/w/api.php",{"action":"sitematrix","formatversion":2,"format":"json","maxage":3600,"smaxage":3600})
sitematrix = response.json()["sitematrix"]

In [105]:
projname_re = re.compile(r"https?://(.*).org")

In [106]:
mapping = {}
rev_mapping = {}
for _,v in sitematrix.items():
    try: 
        site = v['site']
        dbname = site[0]['dbname']
        url = site[0]['url']
        projname = projname_re.findall(url)[0]
        mapping[projname]=dbname
        rev_mapping[dbname]=projname
    except Exception as e:
        continue

In [107]:
spark_mapping = f.create_map([f.lit(x) for x in chain(*mapping.items())])

In [108]:
pageviews = spark.read.table("wmf.projectview_hourly")

pageviews = pageviews.filter(pageviews.agent_type == 'user').select(['project','year','month','day','hour','view_count'])
pageviews = pageviews.withColumn("date", f.concat_ws("-",pageviews.year,pageviews.month,pageviews.day))
pageviews = pageviews.groupBy(['project',"date"]).agg(f.sum('view_count').alias("view_count"))
pageviews = pageviews.withColumn("week", f.date_trunc('week',pageviews.date))
pageviews = pageviews.withColumn("wiki_db",spark_mapping.getItem(pageviews.project))

In [109]:
#missing_in_mapping = pageviews.filter(f.isnull(f.col("wiki_db"))).select(['project','wiki_db']).distinct().collect()

In [110]:
wiki_weeks = wiki_weeks.join(pageviews,on=['wiki_db','week'],how='left_outer')

In [111]:
#wiki_weeks.filter(f.col("wiki_db") == "enwiki").show()

In [112]:
missing_views = wiki_weeks.groupBy('wiki_db').agg(f.mean(f.isnull(f.col("view_count")).cast(IntegerType())).alias("p_no_views"))

In [113]:
# missing_views = missing_views.filter(f.col("p_no_views") == 1).toPandas()

In [114]:
week0 = wmhist.agg(f.min("week")).collect()[0]['min(week)']

In [115]:
page_history = spark.read.table("wmf.mediawiki_page_history")
page_history = page_history.filter(f.col("snapshot") == "2019-04")
# we only need revisions during our time period
# let's assume that pages don't get deleted for now
page_history = page_history.withColumn("page_creation_week",f.date_trunc("week","page_creation_timestamp"))

In [116]:
n_pages_baseline = page_history.filter(
    (page_history.page_creation_week < min_time)) \
.groupBy(['wiki_db']) \
.pivot("page_namespace",interesting_namespaces) \
.count().alias("n_pages_baseline")

cols = ["wiki_db"]
cols.extend("n_pages_baseline_ns_{0}".format(i) for i in n_pages_baseline.columns[1:])
n_pages_baseline = n_pages_baseline.toDF(* cols)

In [117]:
n_new_pages_by_week = page_history \
.filter((page_history.page_creation_week >= min_time)) \
.groupBy(["wiki_db","page_creation_week"]) \
.pivot("page_namespace",interesting_namespaces)\
.agg(f.count("page_id"))

In [118]:
cols = ["wiki_db",'page_creation_week']
cols.extend("n_pages_created_ns_{0}".format(i) for i in n_new_pages_by_week.columns[2:])
n_new_pages_by_week = n_new_pages_by_week.toDF(* cols)

n_pages = n_new_pages_by_week.join(n_pages_baseline, on=['wiki_db'], how='full_outer')

#n_pages = n_pages.withColumn("n_pages_ns",n_pages.n_pages_baseline + n_pages.n_created_pages)
n_pages = n_pages.withColumnRenamed('page_creation_week','week')

In [119]:
wiki_weeks = wiki_weeks.join(n_pages,on=['wiki_db','week'],how='full_outer')

In [120]:
# newcomer survival



In [121]:
wiki_weeks = wiki_weeks.withColumn("year",f.year(f.col("week")))
wiki_weeks = wiki_weeks.withColumn("month",f.month(f.col("week")))

In [122]:
by_wiki_week = wmhist.groupBy('wiki_db','week').agg(f.sum(f.col("is_undo").cast("Int")).alias("n_undos"),
                                                          f.sum(f.col("is_rollback").cast("Int")).alias( "n_rollbacks"))

In [123]:
wiki_weeks = wiki_weeks.join(by_wiki_week, on=['wiki_db','week'])


In [124]:
wiki_weeks.groupby("wiki_db").agg(f.mean( (f.col("n_reverts") > 10).cast(IntegerType()) ).alias("p_30_in_week")).filter(f.col("p_30_in_week") > 0.9)

DataFrame[wiki_db: string, p_30_in_week: double]

In [125]:
conf.getAll()

[('spark.shuffle.file.buffer', '1mb'),
 ('spark.stage.maxConsecutiveAttempts', '10'),
 ('spark.shuffle.service.index.cache.size', '2048'),
 ('spark.shuffle.service.enabled', 'true'),
 ('spark.rpc.io.serverThreads', '64'),
 ('spark.io.compression.lz4.blockSize', '512kb'),
 ('spark.file.transferTo', 'false'),
 ('spark.executorEnv.LD_LIBRARY_PATH', '/usr/lib/hadoop/lib/native'),
 ('spark.unsafe.sorter.spill.reader.buffer.size', '1mb'),
 ('spark.yarn.archive', 'hdfs:///user/spark/share/lib/spark2-assembly.zip'),
 ('spark.sql.shuffle.partitions', '400'),
 ('spark.driver.blockManager.port', '13000'),
 ('spark.shuffle.unsafe.file.output.buffer', '5mb'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.ui.port', '4040'),
 ('spark.yarn.executor.memoryOverhead', '6500m'),
 ('spark.master', 'yarn'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.dynamicAllocation.cachedExecutorIdleTimeout', '3600s'),
 ('spark.yarn.isPython', 'true'),
 ('spark.dynamicAllocation.executorIdleTimeout', '60s'),


In [ ]:
wiki_weeks_out = wiki_weeks.repartition(1)

In [ ]:
wiki_weeks_out.write

In [ ]:
wiki_weeks.select("wiki_db").distinct().collect()

In [ ]:
wiki_weeks.filter(f.col("wiki_db")=='enwiki').select(['other_huggle_N_reverts','week']).show()

In [ ]:
wiki_weeks.write.parquet("/user/nathante/ores_bias/wiki_weeks.parquet",partitionBy=["wiki_db","year","month"],mode='overwrite')

In [ ]:
spark.catalog.setCurrentDatabase("nathante")

In [ ]:
spark.catalog.listTables()

In [ ]:
wiki_weeks = spark.read.parquet("/user/nathante/ores_bias/wiki_weeks.parquet")

In [ ]:
import pyarrow as pa
import pyarrow.parquet as pq

In [ ]:

import os 
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-8-openjdk-amd64'


In [ ]:
hdfs = pa.hdfs.connect()

# hdfs_file = pa.HdfsFile("/user/nathante/ores_bias/wiki_weeks.parquet")

In [ ]:
wiki_weeks_out

In [ ]:
dataset = pq.ParquetDataset("/user/nathante/ores_bias/wiki_weeks.parquet",filesystem=hdfs)

In [ ]:
?pd.read_parquet(dataset)

In [ ]:
# try pulling just the columns you really need. 

In [ ]:
ww_pdf = dataset.read_pandas()

In [ ]:
file = hdfs.read_parquet("/user/nathante/ores_bias/wiki_weeks.parquet")

In [ ]:
dataset = pq.ParquetDataset("/user/nathante/ores_bias/wiki_weeks.parquet")

In [ ]:
ww_pddf = wiki_weeks_out.toPandas()

In [ ]:
ww_pdf.to_csv("ores_bias_data/wiki_weeks2.csv",index=False)